# 用每日新闻预测金融市场变化（标准版）

TF-IDF + SVM 是文本分类问题的基准线

这篇教程我直接用最简单直接的方式处理。

高级版本的教程会在日后的课程中放出。

In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from datetime import date

### 监视数据

我们先读入数据。这里我提供了一个已经combine好了的数据。

In [55]:
data = pd.read_csv('../input/Combined_News_DJIA.csv')

这时候，我们可以看一下数据长什么样子

In [56]:
data.head()

Date  Label                                               Top1  \
0  2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
1  2008-08-11      1  b'Why wont America and Nato help us? If they w...   
2  2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
3  2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
4  2008-08-14      1  b'All the experts admit that we should legalis...   

                                                Top2  \
0            b'BREAKING: Musharraf to be impeached.'   
1        b'Bush puts foot down on Georgian conflict'   
2                 b"Russia 'ends Georgia operation'"   
3  b"When the president ordered to attack Tskhinv...   
4  b'War in South Osetia - 89 pictures made by a ...   

                                                Top3  \
0  b'Russia Today: Columns of troops roll into So...   
1  b"Jewish Georgian minister: Thanks to Israeli ...   
2  b'"If we had no sexual harassment we would hav...   
3  b' Israel clears troops who killed Reuters cam...   
4  b'Swedish wrestler Ara Abrahamian throws away ...   

                                                Top4  \
0  b'Russian tanks are moving towards the capital...   
1  b'Georgian army flees in disarray as Russians ...   
2  b"Al-Qa'eda is losing support in Iraq because ...   
3  b'Britain\'s policy of being tough on drugs is...   
4  b'Russia exaggerated the death toll in South O...   

                                                Top5  \
0  b"Afghan children raped with 'impunity,' U.N. ...   
1      b"Olympic opening ceremony fireworks 'faked'"   
2  b'Ceasefire in Georgia: Putin Outmaneuvers the...   
3  b'Body of 14 year old found in trunk; Latest (...   
4  b'Missile That Killed 9 Inside Pakistan May Ha...   

                                                Top6  \
0  b'150 Russian tanks have entered South Ossetia...   
1  b'What were the Mossad with fraudulent New Zea...   
2  b'Why Microsoft and Intel tried to kill the XO...   
3  b'China has moved 10 *million* quake survivors...   
4  b"Rushdie Condemns Random House's Refusal to P...   

                                                Top7  \
0  b"Breaking: Georgia invades South Ossetia, Rus...   
1  b'Russia angered by Israeli military sale to G...   
2  b'Stratfor: The Russo-Georgian War and the Bal...   
3  b"Bush announces Operation Get All Up In Russi...   
4  b'Poland and US agree to missle defense deal. ...   

                                                Top8  \
0  b"The 'enemy combatent' trials are nothing but...   
1  b'An American citizen living in S.Ossetia blam...   
2  b"I'm Trying to Get a Sense of This Whole Geor...   
3             b'Russian forces sink Georgian ships '   
4  b'Will the Russians conquer Tblisi? Bet on it,...   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                                               Top16  \
0  b'Georgia Invades South Ossetia - if Russia ge...   
1  b'Israel and the US behind the Georgian aggres...   
2  b'U.S. troops still in Georgia (did you know t...   
3                      b'Elephants extinct by 2020?'   
4  b'Bank analyst forecast Georgian crisis 2 days...   

                                               Top17  \
0                b'Al-Qaeda Faces Islamist Backlash'   
1  b'"Do not believe TV, neither Russian nor Geor...   
2       b'Why Russias response to Georgia was right'   
3  b'US humanitarian missions soon in Georgia - i...   
4  b"Georgia confict could set back Russia's US r...   

                                               Top18  \
0  b'Condoleezza Rice: "The US would not act to p...   
1  b'Riots are still going on in Montreal (Canada...   
2  b'Gorbachev accus

其实看起来特别的简单直观。如果是1，那么当日的DJIA就提高或者不变了。如果是1，那么DJIA那天就是跌了。

接下来我们把headlines先合并起来。因为我们显然是需要考虑所有的news的。

In [57]:
data["combined_news"] = data.filter(regex=("Top.*")).apply(lambda x: ''.join(str(x.values)), axis=1)

### 分割测试/训练集

这下，我们可以把数据给分成Training/Testing data

In [58]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

### 提取features

也就是文本中的特征
这里一定要注意，fit你的model的时候，要用training set，不能一股脑的把所有的数据都放进来。（当然，现实中你可以这么做）因为我们要假设testing set我们在训练的时候是完全接触不到的，是不可知的。

In [31]:
feature_extraction = TfidfVectorizer()
X_train = feature_extraction.fit_transform(train["combined_news"].values)

所以*X_train* 搞完以后，直接给 *X_test* 做个Transform

In [32]:
X_test = feature_extraction.transform(test["combined_news"].values)

同理，y就是我们已经准备好的label

In [33]:
y_train = train["Label"].values
y_test = test["Label"].values

### 训练模型

In [34]:
clf = SVC(probability=True, kernel='rbf')

把你的*X_train*和*y_train*给fit进去

In [35]:
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### 预测

In [36]:
predictions = clf.predict_proba(X_test)

### 验证准确度

按照我之前给的要求，用AUC作为binary classification的Metrics

In [37]:
print('ROC-AUC yields ' + str(roc_auc_score(y_test, predictions[:,1])))

ROC-AUC yields 0.574260752688


## 进阶版

### 文本预处理

我们这样直接把文本放进TF-IDF，虽然简单方便，但是还是不够严谨的。
我们可以把原文本做进一步的处理。

+ 小写 / 分成小tokens

In [59]:
X_train = train["combined_news"].str.lower().str.replace('"', '').str.replace("'", '').str.split()
X_test = test["combined_news"].str.lower().str.replace('"', '').str.replace("'", '').str.split()
print(X_test[1611])

['[', 'most', 'cases', 'of', 'cancer', 'are', 'the', 'result', 'of', 'sheer', 'bad', 'luck', 'rather', 'than', 'unhealthy', 'lifestyles,', 'diet', 'or', 'even', 'inherited', 'genes,', 'new', 'research', 'suggests.', 'random', 'mutations', 'that', 'occur', 'in', 'dna', 'when', 'cells', 'divide', 'are', 'responsible', 'for', 'two', 'thirds', 'of', 'adult', 'cancers', 'across', 'a', 'wide', 'range', 'of', 'tissues.', 'iran', 'dismissed', 'united', 'states', 'efforts', 'to', 'fight', 'islamic', 'state', 'as', 'a', 'ploy', 'to', 'advance', 'u.s.', 'policies', 'in', 'the', 'region:', 'the', 'reality', 'is', 'that', 'the', 'united', 'states', 'is', 'not', 'acting', 'to', 'eliminate', 'daesh.', 'they', 'are', 'not', 'even', 'interested', 'in', 'weakening', 'daesh,', 'they', 'are', 'only', 'interested', 'in', 'managing', 'it', 'poll:', 'one', 'in', '8', 'germans', 'would', 'join', 'anti-muslim', 'marches', 'uk', 'royal', 'familys', 'prince', 'andrew', 'named', 'in', 'us', 'lawsuit', 'over', 'un

+ 删减停止词

In [60]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

+ 删除数字

In [61]:
import re
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

+ lemma

In [62]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

我们把这些元素全都合成一个func

In [63]:
def check(word):
    """
    如果需要这个单词，则True
    如果应该去除，则False
    """
    if word in stop:
        return False
    elif hasNumbers(word):
        return False
    else:
        return True

然后我们把整个流程放进我们的DF中处理

In [64]:
X_train = X_train.apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x if check(item)])
X_test = X_test.apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x if check(item)])
print(X_test[1611])

['[', 'case', 'cancer', 'result', 'sheer', 'bad', 'luck', 'rather', 'unhealthy', 'lifestyles,', 'diet', 'even', 'inherited', 'genes,', 'new', 'research', 'suggests.', 'random', 'mutation', 'occur', 'dna', 'cell', 'divide', 'responsible', 'two', 'third', 'adult', 'cancer', 'across', 'wide', 'range', 'tissues.', 'iran', 'dismissed', 'united', 'state', 'effort', 'fight', 'islamic', 'state', 'ploy', 'advance', 'u.s.', 'policy', 'region:', 'reality', 'united', 'state', 'acting', 'eliminate', 'daesh.', 'even', 'interested', 'weakening', 'daesh,', 'interested', 'managing', 'poll:', 'one', 'german', 'would', 'join', 'anti-muslim', 'march', 'uk', 'royal', 'family', 'prince', 'andrew', 'named', 'u', 'lawsuit', 'underage', 'sex', 'allegation', 'asylum-seekers', 'refused', 'leave', 'bus', 'arrived', 'destination', 'rural', 'northern', 'sweden,', 'demanding', 'taken', 'back', 'malm', 'big', 'city.', 'pakistani', 'boat', 'blow', 'self', 'india', 'navy', 'chase.', 'four', 'people', 'board', 'vessel',

因为外部库，比如sklearn 只支持string输入，所以我们把调整后的list再变回string

In [65]:
X_train = X_train.apply(lambda x: ' '.join(x))
X_test = X_test.apply(lambda x: ' '.join(x))
print(X_test[1611])

[ case cancer result sheer bad luck rather unhealthy lifestyles, diet even inherited genes, new research suggests. random mutation occur dna cell divide responsible two third adult cancer across wide range tissues. iran dismissed united state effort fight islamic state ploy advance u.s. policy region: reality united state acting eliminate daesh. even interested weakening daesh, interested managing poll: one german would join anti-muslim march uk royal family prince andrew named u lawsuit underage sex allegation asylum-seekers refused leave bus arrived destination rural northern sweden, demanding taken back malm big city. pakistani boat blow self india navy chase. four people board vessel near pakistani port city karachi believed killed dramatic episode arabian sea new year eve, according india defence ministry. sweden hit third mosque arson attack week car set alight french new year salary top ceo rose twice fast average canadian since recession: study norway violated equal-pay law, ju

重新Fit一遍我们的clf

In [66]:
feature_extraction = TfidfVectorizer(lowercase=False)
X_train = feature_extraction.fit_transform(X_train.values)
X_test = feature_extraction.transform(X_test.values)

再跑一遍

In [67]:
clf = SVC(probability=True, kernel='rbf')
clf.fit(X_train, y_train)
predictions = clf.predict_proba(X_test)
print('ROC-AUC yields ' + str(roc_auc_score(y_test, predictions[:,1])))

ROC-AUC yields 0.539566532258


不小心发现，折腾一圈以后，这个结果还不如之前的简单版。

这个故事告诉我们，**逼装得太多，不一定有卵用。**

造成如此的原因有几种：

+ 数据点太少

在大量的数据下，标准的文本预处理流程还是需要的，以提高机器学习的准确度。

+ One-Off result

我们到现在都只是跑了一次而已。如果我们像前面的例子一样，用Cross Validation来玩这组数据，说不定我们会发现，分数高的clf其实是overfitted了的。

所以，我们在做kaggle竞赛的时候，最好是要给自己的clf做好CV验证。不要贻笑大方。
